EX: 1 TRƯỜNG CẤP 3 có 3 cấp 10,11,12 

- Khối 10: 20 lớp
- Khối 11: 18 lớp
- khối 12: 15 lớp
mỗi lớp sẽ có random từ 30-36 mỗi    học sinh sẽ được identify bằng school ID tăng dần từ 1 -> tổng số học sinh. Class ID số thứ tự trong lớp từ 1 -> số học sinh một lớp. Mỗi học sinh có 1 aray chứa điểm theo thứu tự:
toán, lý , hóa , ngữ văn , sử , địa, anh văn, công nghệ, GDCD, tin học



------ require generator------
thư mục lớn: trường
trong thư lớn có 3 thư mục nhỏ :3 khối
trong thư mục nhỏ có file theo tên lớp: 10A1.js,..... -> bên trong chứa Array Json : chứa từng json là từng học sinh trong lớp HS:{SCHOOL ID : INT, CLASS ID:INT, SCORES:[average theo định nghĩa random theo Float ]} 
~70% học sinh phải có điểm TB môn > TB , ~30% học sinh có điểm dưới trung bình 


----- require read file -----
- Xuất ra 1 cái folder y chang trên chứa them rank của từng khối với điểm học sinh và loại học sinh

- Tổng số học sinh giỏi ,tổng số học sinh khá, tổng học sinh trung bình tạo ra 1 file thống kê

- Xuất ra 3 file chứa tổng học sinh giỏi , tổng học sinh khá, tổng học sinh trung bình từng khối

- Lọc ra top 3 học sinh giỏi nhất của từng khối

In [32]:
# library
from functools import *
from pathlib import Path
import json
import os
import random

In [2]:
class data_generator:
   def __init__(self,school = "",grade_class = {},parent_dir = ''):
      self.school = school
      self.grade_class = grade_class
      self.parent_dir = parent_dir
      self.ID_school = 0

   def create_foler(self):
      self.school = "Le Quy Don"
      self.grade_class = {"10":20, "11":18, "12":15}
      self.parent_dir = "../Generator/Data"
      for key,val in self.grade_class.items():
         # Path
         path = os.path.join(self.parent_dir, key)
         # Create the directory
         try:
            os.makedirs(path, exist_ok = True)
            print("Directory '%s' created successfully" % key)
         except OSError as error:
            print("Directory '%s' can not be created" % key)
      
   def generate_data_class(self):
      self.ID_school += 1
      if not bool(self.grade_class):
         return None
      grade,ordinal_numbers = random.choice(list(self.grade_class.items()))
      if ordinal_numbers > 0:
         student_total = random.randint(30,35)
         self.grade_class[grade] = ordinal_numbers -1
         for i in range(0,student_total+1):
            percentage_avr = random.random()
            overall = lambda x: random.uniform(0,5) if percentage_avr < 0.3 else random.uniform(5,10)
            avr_score = overall(percentage_avr)
            score = {"Math":avr_score, "Physic":avr_score, "Chemistry": avr_score, "Literature":avr_score, "History": avr_score, "Geographic": avr_score,
                     "English": avr_score, "Technique": avr_score, "Common Sense": avr_score, "Computer": avr_score}
            sub_higher = 10.0 - avr_score
            sub_lower = avr_score - 10.0
            while(sub_higher != 0 or sub_lower != 0 ):
               alpha = random.random()
               beta = random.random()
               subject,subj_score = random.choice(list(score.items()))
               subj_score = subj_score + alpha*sub_higher + beta*sub_lower
               sub_higher = sub_higher - alpha*sub_higher
               sub_lower = sub_lower - beta*sub_lower
               score[subject] = subj_score
            ## append ordinal_numbers to file
            final = False
            if i == student_total:
               final = True
            value_append_file = {"School ID": self.ID_school, "Class ID": i, "Score": score }
            self.dump_data_to_file(grade,ordinal_numbers,value_append_file, final)
      else:
         del self.grade_class[grade]
      return self.generate_data_class()          
   def dump_data():
      pass
   def control_generator():
      pass

   def dump_data_to_file(self, grade, ordinal_numbers, value, final):
      filename = Path(str(grade) + "/" + str(grade) + "A" + str(ordinal_numbers) + ".json")
      fileToOpen = "Data"/filename
      if (os.path.exists(fileToOpen) == False):
         with open(fileToOpen,"a") as file:
            file.write('[')
            json.dump(value, file, indent = 4)
      else:
         with open(fileToOpen,"a+") as file:
            file.write(',')
            json.dump(value, file, indent = 4)
         if final:
            with open(fileToOpen,"a+") as file:
               file.write(']')

generator = data_generator()
generator.create_foler()
generator.generate_data_class()    
print(generator.grade_class)

Directory '10' created successfully
Directory '11' created successfully
Directory '12' created successfully
{}


In [5]:
# Checking a ability
def ranking_student(student):
    overall = sum(student["Score"].values())/10
    good = all(v > 6.5 for v in student["Score"].values()) and (student["Score"]["Math"]>6.5 or student["Score"]["Literature"]>6.5 )
    medium = all(v > 5 for v in student["Score"].values()) and (student["Score"]["Math"]>6.5 or student["Score"]["Literature"]>6.5 )
    weak = all(v > 3.5 for v in student["Score"].values()) and (student["Score"]["Math"]>6.5 or student["Score"]["Literature"]>6.5 )
    if overall > 8 and good:
        student["rank"] = "good"
    elif overall > 6.5 and medium:
        student["rank"] = "medium"   
    else: 
        student["rank"] = "weak"
    student["average"] = overall
    return student

def Edit_student_rank():
    students_rank = {
        "good": 0
        ,"medium": 0
        ,"weak": 0
    }
    path = "Data"
    dir_list = os.listdir(path)
    for i in dir_list:
        path_folder = path+"/"+i
        folder_list = os.listdir(path_folder)
        for j in folder_list:
            file_path = Path(path_folder + "/" + j)
            with open(file_path) as file:
                data = json.load(file)
            data = list(map(ranking_student,data))
            with open(file_path,"w+") as file:
                json.dump(data,file,indent=4)

Edit_student_rank()
%timeit Edit_student_rank()

195 ms ± 66.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
def Count_student_rank ():
    students_rank = {
        "good": 0
        ,"medium": 0
        ,"weak": 0
    }
    path = "Data"
    dir_list = os.listdir(path)
    for i in dir_list:
        path_folder = path+"/"+i
        folder_list = os.listdir(path_folder)
        for j in folder_list:
            file_path = path_folder + "/" + j
            with open(file_path) as file:
                data = json.load(file) 
                data = [students_rank] + data  
            students_rank = reduce(increase_ranks, data)
    return students_rank

def increase_ranks(x, y):
    for item in x.keys():
        if y["rank"] == item:
            x[item] = x[item] + 1
    return x
count_student = Count_student_rank()
print(count_student)
%timeit Count_student_rank()

{'good': 497, 'medium': 356, 'weak': 931}
21.1 ms ± 8.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
# run command line
! pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [38]:
#Xuất ra 3 file chứa tổng học sinh giỏi , tổng học sinh khá, tổng học sinh trung bình từng khối
def stastic_collecting():
    path = "Data"
    dir_list = os.listdir(path)
    for i in dir_list:
        path_folder = path+"/"+i
        folder_list = os.listdir(path_folder)
        students = {
        "good": []
        ,"medium": []
        ,"weak": []
         }
        for j in folder_list:
            file_path = path_folder + "/" + j
            with open(file_path) as file:
                data = json.load(file) 
                data = [students] + data
            students = reduce(increase_stastics, data)
            for item in students.items():
                result_path = path_folder + '/' + item[0]
                with open(result_path, "w+") as file:
                    json.dump(item[1],file,indent=4)
def increase_stastics(x, y):
    rank = y["rank"]
    x[rank].append(y)
    return x

stastic_collecting()            

In [1]:
def stastic_collecting_best():
    path = "Data"
    dir_list = os.listdir(path)
    students_ranking = {
        "10": []
        ,"11": []
        ,"12": []
         }
    for i in dir_list:
        path_file = path + '/' + i + "/good"
        with open(path_file) as file:
            data = json.load(file)
        data.sort(key = lambda x: x['average'], reverse=True)
        students_ranking[i].append(data[:3])
    return students_ranking

stastic_collecting_best()

NameError: name 'json' is not defined